In [1]:
import torch
from det3d.torchie import Config
from det3d.models import build_detector
from det3d.torchie.trainer import load_checkpoint

config = "/workspace/centerformer/configs/nusc/nuscenes_centerformer_poolformer.py"

cfg = Config.fromfile(config)
checkpoint_path = "/workspace/centerformer/work_dirs/nuscenes_poolformer/poolformer.pth"

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu")

Deformable Convolution not built!
Deformable Convolution not built!
Use HM Bias:  -2.19


/data/centerformer/det3d/core/bbox/geometry.py:149: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:162: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:280: NumbaDeprecationWarning: The 'nopython' keyword argument was

In [2]:
from torch import nn

class CenterFinder(nn.Module):
    def __init__(self, model):
        super(CenterFinder, self).__init__()
        self.model = model

    def forward(self, x):
        return self.model.neck.find_centers(x) # pos_features, out_dict_list, ct_feat 
    
centerFinder = CenterFinder(model)
centerFinder.cuda()
centerFinder.eval();

In [3]:
import pickle
pickle_dir = "/workspace/centerformer/work_dirs/partition/sample_data/"

with open(pickle_dir + "findcenter_input.pkl", 'rb') as handle:
    x = pickle.load(handle)
        
with open(pickle_dir + "findcenter_output1.pkl", 'rb') as handle:
    findcenter_output1 = pickle.load(handle)
with open(pickle_dir + "findcenter_output2.pkl", 'rb') as handle:
    findcenter_output2 = pickle.load(handle)
with open(pickle_dir + "findcenter_output3.pkl", 'rb') as handle:
    findcenter_output3 = pickle.load(handle)

In [4]:
with torch.no_grad():
    ct_feat, center_pos_embedding, out_scores, out_labels, out_order, out_mask = centerFinder(x)

/root/anaconda3/envs/centerformer/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [5]:
import tensorrt as trt
trt.__version__

'8.5.1.7'

In [6]:
centerFinder_engine_path = 'findCenter_folded.trt'
        
def load_engine(engine_filepath, trt_logger):
    with open(engine_filepath, "rb") as f, trt.Runtime(trt_logger) as runtime:
        engine = runtime.deserialize_cuda_engine(f.read())
    return engine

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
centerFinder_engine = load_engine(centerFinder_engine_path, TRT_LOGGER)
centerFinder_context = centerFinder_engine.create_execution_context()

[12/13/2023-07:12:39] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars


In [7]:
def run_trt_engine(context, engine, tensors):
    bindings = [None]*engine.num_bindings
    for idx, binding in enumerate(engine):
        tensor_name = engine.get_tensor_name(idx)
        if engine.get_tensor_mode(binding)==trt.TensorIOMode.INPUT:
            bindings[idx] = tensors['inputs'][tensor_name].data_ptr()
            if context.get_tensor_shape(tensor_name):
                context.set_input_shape(tensor_name, tensors['inputs'][tensor_name].shape)
        else:
            bindings[idx] = tensors['outputs'][tensor_name].data_ptr()
    context.execute_v2(bindings=bindings)

In [8]:
ct_feat_tensor = torch.zeros((1, 3000, 256)).to(x.device)
center_pos_embedding_tensor = torch.zeros((1, 3000, 256)).to(x.device)
out_scores_tensor = torch.zeros((6, 1, 500)).to(x.device)
out_labels_tensor = torch.zeros((6, 1, 500)).to(x.device)
out_order_tensor = torch.zeros((6, 1, 500)).to(x.device)
out_mask_tensor = torch.zeros((6, 1, 500)).to(x.device)

IO_tensors = {
                "inputs" :
                {'input_tensor': x},
                "outputs" :
                {'ct_feat': ct_feat_tensor, 'center_pos_embedding': center_pos_embedding_tensor,
                'out_scores': out_scores_tensor, 'out_labels': out_labels_tensor,
                'out_order': out_order_tensor, 'out_mask': out_mask_tensor}
            }

In [9]:
run_trt_engine(centerFinder_context, centerFinder_engine, IO_tensors)

In [12]:
print(ct_feat_tensor[0][0][:10])
print(ct_feat[0][0][:10])

torch.Size([1, 3000, 256])